# Day 7 - Lab 1: Agent Interoperability with the Model Context Protocol (MCP)

**Objective:** Build a LangChain agent that can use tools exposed by a standard MCP server, demonstrating how MCP provides a universal adapter for AI tool integration.

**Introduction:**
For AI agents to be truly useful, they need to interact with the outside world: files, databases, and APIs. But how can an agent work with hundreds of different tools without custom code for each one? The Model Context Protocol (MCP) is an open standard designed to solve this problem. It acts as a universal adapter.

In this lab, you will connect to a (mock) MCP server that provides tools for interacting with a filesystem. You will then use the `langchain-mcp` adapter library to seamlessly integrate these external tools into a LangChain agent.

## Step 1: Setup

We'll install the necessary libraries and set up our LangChain LLM client.

In [ ]:
!pip install -q mcp langchain-mcp langchain_openai
import sys, os, json, datetime
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_mcp import create_mcp_tools
from langchain_core.tools import Tool

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")
llm = ChatOpenAI(model=model_name, temperature=0)

## Step 2: Simulating an MCP Server

In a real-world scenario, an MCP server would be a separate, long-running process. To keep this lab self-contained, we will simulate one. This `MockMCPSession` class mimics a connection to an MCP server that offers two tools: `read_file` and `get_system_time`.

In [ ]:
class MockMCPSession:
    """A mock MCP client session to simulate a connection to an MCP server."""
    async def list_tools(self):
        print("--> Mock MCP Server: list_tools() called")
        return [
            {
                'name': 'read_file',
                'description': 'Reads the content of a specified file.',
                'inputSchema': {
                    'type': 'object',
                    'properties': {
                        'path': {'type': 'string', 'description': 'The path to the file.'}
                    },
                    'required': ['path']
                }
            },
            {
                'name': 'get_system_time',
                'description': 'Returns the current system time.',
                'inputSchema': {'type': 'object', 'properties': {}}
            }
        ]

    async def call_tool(self, name, arguments):
        print(f"--> Mock MCP Server: call_tool('{name}') called with {arguments}")
        if name == 'read_file':
            # For safety, we'll only allow reading the README.md
            if arguments.get('path') == 'README.md':
                try:
                    with open(os.path.join(project_root, 'README.md'), 'r') as f:
                        return f.read()
                except FileNotFoundError:
                    return 'Error: README.md not found.'
            else:
                return f"Error: Access denied to file: {arguments.get('path')}"
        elif name == 'get_system_time':
            return datetime.datetime.now().isoformat()
        else:
            return 'Error: Tool not found'

mock_session = MockMCPSession()
print("Mock MCP Server Session created.")

## Step 3: The Challenges

### Challenge 1 (Foundational): Discover and List MCP Tools

**Task:** Use the `mock_session` to discover the tools offered by the MCP server and print their details.

**Instructions:**
1. Call the `list_tools()` method on the `mock_session` object.
2. Loop through the results and print the name and description of each available tool.

In [ ]:
import asyncio

async def list_mcp_tools():
    print("--- Discovering MCP Tools ---")
    # TODO: Call the list_tools method on the mock_session
    available_tools = [] # Your code here

    # TODO: Loop through available_tools and print the name and description of each tool
    # Your code here

# In a notebook, we can run async code like this:
await list_mcp_tools()

### Challenge 2 (Intermediate): Convert MCP Tools to LangChain Tools

**Task:** Use the `create_mcp_tools` function from the `langchain-mcp` library to automatically convert the MCP tool definitions into a list of LangChain `Tool` objects.

**Instructions:**
1. Call the `create_mcp_tools` function, passing it our `mock_session`.
2. This function is asynchronous, so you will need to `await` its result.
3. Store the result in a variable called `langchain_tools`.
4. Print the `langchain_tools` list to see the LangChain-compatible tool objects.

In [ ]:
async def convert_tools():
    print("\n--- Converting MCP tools to LangChain Tools ---")
    # TODO: Call create_mcp_tools with the mock_session
    langchain_tools = [] # Your code here

    print(f"Successfully converted {len(langchain_tools)} tools.")
    print(langchain_tools)
    return langchain_tools

langchain_tools = await convert_tools()

### Challenge 3 (Advanced): Build and Use an Agent with MCP Tools

**Task:** Now that you have a list of standard LangChain tools, build a LangChain agent that can use them to answer questions.

**Instructions:**
1. Create a `ChatPromptTemplate` for your agent. It must include a placeholder for `{agent_scratchpad}`.
2. Use `create_tool_calling_agent` to create the agent, passing the `llm`, the `langchain_tools` from the previous step, and your prompt.
3. Create an `AgentExecutor` to run the agent.
4. Invoke the agent with a question that requires using one of the MCP tools, like `"What is the current system time?"`

In [ ]:
# TODO: 1. Define the prompt template for the agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant with access to system tools."),
    ("user", "{input}"),
    # Your placeholder here
])

# TODO: 2. Create the tool-calling agent
agent = None # Your code here

# TODO: 3. Create the AgentExecutor
agent_executor = None # Your code here

print("\n--- Invoking agent to get system time ---")
# TODO: 4. Invoke the agent with a question about the time
time_result = None # Your code here
print(f"Final Answer: {time_result['output']}")

print("\n--- Invoking agent to read a file ---")
# Bonus: Invoke the agent again to read the README.md file
file_result = agent_executor.invoke({"input": "What is the content of the README.md file?"})
print(f"Final Answer: {file_result['output']}")

## Lab Conclusion

Congratulations! You have successfully used the Model Context Protocol to integrate external tools into a LangChain agent. You saw how MCP provides a standard way for a server to describe its tools, and how the `langchain-mcp` library can automatically discover and adapt these tools for your agent. This is a key architectural pattern for building complex, interoperable multi-agent systems that can securely interact with a wide range of enterprise tools.